In [1]:
test1 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test2 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"

def bracket_filter(sentence):
  new_sentence = str()
  flag = False
  
  for ch in sentence:
    if ch == '(' and flag == False:
      flag = True
      continue
    if ch == '(' and flag == True:
      flag = False
      continue
    if ch != ')' and flag == False:
      new_sentence += ch
  return new_sentence

print(bracket_filter(test1))
print(bracket_filter(test2))


o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/
근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?


In [2]:
import re

test1 = "o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/"
test2 = "c# 배워봤어?"

def special_filter(sentence):
  SENTENCE_MARK = ['?', '!']
  NOISE = ['o', 'n', 'u', 'b', 'l']
  EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', '.', ',']
  
  new_sentence = str()
  for idx, ch in enumerate(sentence):
    if ch not in SENTENCE_MARK:
      # o/, n/ 등 처리
      if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': 
        continue 

    if ch == '#': 
      new_sentence += '샾'

    elif ch not in EXCEPT: 
      new_sentence += ch

  pattern = re.compile(r'\s\s+')
  new_sentence = re.sub(pattern, ' ', new_sentence.strip())
  return new_sentence

print(special_filter(test1))
print(special_filter(test2))


근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸?
c샾 배워봤어?


In [3]:
# test = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test = 'b/ 아 나 (3DS)/(쓰리 DS) 갖고 싶다. (3DS)/(쓰리 DS)'

def sentence_filter(raw_sentence):
  return special_filter(bracket_filter(raw_sentence))

print(sentence_filter(test))


아 나 쓰리 DS 갖고 싶다 쓰리 DS


In [4]:
chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung])
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong])

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

class Phoneme:
  def __init__ (self):
    self.BLANK_INDEX = 0
    self.START_INDEX = 1
    self.END_INDEX = 2
    self.SPACE_INDEX = 3
    self.OOV_INDEX = 4

class PhonemeGenerator (Phoneme):
  def __init__ (self, labels):
    super(PhonemeGenerator, self).__init__()
    self.phonemes = ['<BLANK>', '<START>', '<END>', ' ']

    texts = []
    for label in labels:
      for phoneme in label:
        if not isHangeul(phoneme) and not phoneme == ' ' and not phoneme == '?':
          break
        for pure in pureosseugi(phoneme):
          texts.append(pure)

    tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)

    for i in range(1, len(tokenizer.index_word) + 1):
      self.phonemes.append(tokenizer.index_word[i])

    self.phoneme_index = {}
    for i, phoneme in enumerate(self.phonemes):
      self.phoneme_index[phoneme] = i
    
    self.VOCAB_SIZE = len(self.phonemes)
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phonemes, self.phoneme_index), f)


class PhonemeLoader (Phoneme):
  def __init__ (self, filename):
    super(PhonemeLoader, self).__init__()
    with open(filename, 'rb') as f:
      self.phonemes, self.phoneme_index = pickle.load(f)

    self.VOCAB_SIZE = len(self.phonemes)


In [6]:
import random
import os
import numpy as np
import librosa

class Dataset:
  def get_data (self, idx):
    return (
      self.extract_spectrogram(self.file_phoneme_vector_pairs[idx][0]),
      [0] + self.file_phoneme_vector_pairs[idx][1]
    )
  
  def get_len (self):
    return len(self.file_phoneme_vector_pairs)

  def extract_spectrogram (self, filename):
    waveform = np.memmap(filename, dtype='h', mode='r')

    sr = 16000
    st = 1 / sr

    t = np.linspace(0, waveform.shape[0] * st, waveform.shape[0])

    S = librosa.feature.melspectrogram(y=np.float32(waveform), n_fft=512, hop_length=256, window='hamming', sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    log_S = np.transpose(log_S)

    return log_S
  
  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.phoneme, self.file_phoneme_vector_pairs), f)

class RandomDataset (Dataset):
  def __init__ (self):
    super(RandomDataset, self).__init__()
    with open('./ko-audio-dataset/sentence-script/KsponSpeech_scripts/train.trn', 'r', encoding='utf-8') as file:
      lines = file.readlines()
    def process_line (line):
      line = line.split(' :: ')
      line[0] = os.path.join('./ko-audio-dataset/audio/', line[0])
      line[1] = sentence_filter(line[1])
      return line
    file_label_pairs = list(map(process_line, lines))
    file_label_pairs = list(filter(lambda pair: not self.check_skip_label(pair[1]), file_label_pairs))
    self.phoneme = PhonemeGenerator([label for file, label in file_label_pairs])
    file_phoneme_pairs = list(map(lambda pair: (pair[0], pureosseugi(pair[1])), file_label_pairs))
    self.file_phoneme_vector_pairs = [[pair[0], [self.phoneme.phoneme_index[phoneme] for phoneme in pair[1]]] for pair in file_phoneme_pairs]
    random.shuffle(self.file_phoneme_vector_pairs)
  
  def check_skip_hangul (self, hangul):
    return not isHangeul(hangul) and not hangul == ' ' and not hangul == '?'

  def check_skip_label (self, label):
    for hangul in label:
      if self.check_skip_hangul(hangul):
        return True
    return False

class LoadedDataset (Dataset):
  def __init__ (self, filename):
    super(LoadedDataset, self).__init__()
    with open(filename, 'rb') as f:
      self.phoneme, self.file_phoneme_vector_pairs = pickle.load(f)



c:\Users\akswnd98\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [7]:
class DatasetSizeLimitingIterator:
  def __init__ (self, dataset, time_limit, label_limit, idx):
    self.dataset = dataset
    self.time_limit = time_limit
    self.label_limit = label_limit
    self.idx = idx
    while self.idx < self.dataset.get_len():
      data = self.dataset.get_data(self.idx)
      if data[0].shape[0] <= self.time_limit and len(data[1]) <= self.label_limit:
        break
      self.idx += 1

  def next (self):
    idx = self.idx + 1
    while idx < self.dataset.get_len():
      data = self.dataset.get_data(idx)
      if data[0].shape[0] <= self.time_limit and len(data[1]) <= self.label_limit:
        break
      idx += 1
    return DatasetSizeLimitingIterator(self.dataset, self.time_limit, self.label_limit, idx)
  
  def check_end (self):
    return self.idx >= self.dataset.get_len()

  def get_value (self):
    return self.dataset.get_data(self.idx)

  def save (self, filename):
    with open(filename, 'wb') as f:
      pickle.dump((self.time_limit, self.label_limit,self.idx), f)

class IteratorLoader:
  @staticmethod
  def load (dataset, filename):
    with open(filename, 'rb') as f:
      time_limit, label_limit, idx = pickle.load(f)
    return DatasetSizeLimitingIterator(dataset, time_limit, label_limit, idx)


In [8]:
import os

class DatasetStepExtractorLoader:
  @staticmethod
  def load (path):
    dataset = LoadedDataset(os.path.join(path, 'dataset.pickle'))
    return SavableDatasetStepExtractor(
      dataset,
      pickle.load(open(os.path.join(path, 'step_size.pickle'), 'rb')),
      IteratorLoader.load(dataset, os.path.join(path, 'iterator.pickle'))
    )

class DatasetStepExtractor:
  def __init__ (self, dataset, step_size, iterator):
    self.dataset = dataset
    self.step_size = step_size
    self.value = []
    self.iterator = iterator
    while not iterator.check_end() and len(self.value) < self.step_size:
      self.value.append(iterator.get_value())
      iterator = iterator.next()
    self.is_end = len(self.value) < self.step_size
    self.next_iterator = iterator
  
  def check_end (self):
    return self.is_end
  
  def get_value (self):
    return self.value
  
  def get_next_extractor (self):
    return DatasetStepExtractor(self.dataset, self.step_size, self.next_iterator)

class SavableDatasetStepExtractor (DatasetStepExtractor):
  def save (self, saver):
    saver.save(self.dataset, self.step_size, self.iterator)

class DatasetStepExtractorSaver:
  def __init__ (self, path):
    self.path = path
  
  def save (self, dataset, step_size, iterator):
    os.makedirs(self.path, exist_ok=True)

class IteratorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
  
class DatasetSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    dataset.save(os.path.join(self.path, 'dataset.pickle'))

class StepSizeSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)

class DatasetStepExtractorSaver (DatasetStepExtractorSaver):
  def save (self, dataset, step_size, iterator):
    super().save(dataset, step_size, iterator)
    iterator.save(os.path.join(self.path, 'iterator.pickle'))
    dataset.save(os.path.join(self.path, 'dataset.pickle'))
    with open(os.path.join(self.path, 'step_size.pickle'), 'wb') as f:
      pickle.dump(step_size, f)


In [9]:
import tensorflow as tf

In [10]:
class Encoder (tf.keras.Model):
  def __init__ (self, units, output_dim):
    super(Encoder, self).__init__(self)
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)
    
  def call (self, inputs):
    rnn_outputs, hidden_states = self.gru(inputs)
    outputs = self.fc(rnn_outputs)
    return outputs


In [11]:
class Decoder (tf.keras.Model):
  def __init__ (self, vocab_size, embedding_dim, units, output_dim):
    super(Decoder, self).__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(units, recurrent_initializer='glorot_uniform', return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(output_dim)

  def call (self, inputs):
    embedded = self.embedding(inputs)
    outputs, hidden_states = self.gru(embedded)
    outputs = self.fc(outputs)
    return outputs, hidden_states


In [12]:
class JointNet (tf.keras.Model):
  def __init__ (self, inner_dim, vocab_size):
    super(JointNet, self).__init__()
    self.forward_layer = tf.keras.layers.Dense(inner_dim, activation='tanh')
    self.project_layer = tf.keras.layers.Dense(vocab_size)
  
  def call (self, inputs):
    enc_outputs, dec_outputs = inputs
    joint_inputs = tf.expand_dims(enc_outputs, axis=2) + tf.expand_dims(dec_outputs, axis=1)
    outputs = self.forward_layer(joint_inputs)
    outputs = self.project_layer(outputs)
    return outputs


In [13]:
class Transducer (tf.keras.Model):
  def __init__ (self, encoder, decoder, joint_net):
    super(Transducer, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.joint_net = joint_net

  def __init__ (self, embedding_dim, units, coder_output_dim, joint_net_inner_dim, vocab_size):
    super(Transducer, self).__init__()
    self.encoder = Encoder(units, coder_output_dim)
    self.decoder = Decoder(vocab_size, embedding_dim, units, coder_output_dim)
    self.joint_net = JointNet(joint_net_inner_dim, vocab_size)
    self.vocab_size = vocab_size
  
  def call (self, inputs):
    enc_inputs, dec_inputs = inputs
    enc_outputs = self.encoder(enc_inputs)
    dec_outputs, dec_states = self.decoder(dec_inputs)
    outputs = self.joint_net([enc_outputs, dec_outputs])
    return outputs


In [14]:
class AudioDecoder:
  def __init__ (self, transducer):
    self.transducer = transducer
  
  def decode (self, spectrogram):
    label = [0]
    for i in range(spectrogram.shape[0]):
      sub_spectrogram = spectrogram[0: i + 1]
      sub_spectrogram = tf.convert_to_tensor(sub_spectrogram)
      sub_spectrogram = tf.expand_dims(sub_spectrogram, axis=0)
      sub_label = tf.constant([label], dtype=tf.int32)
      sub_rst = self.transducer((sub_spectrogram, sub_label))
      sub_rst = tf.argmax(sub_rst[0][-1][-1], axis=-1).numpy()
      label.append(sub_rst)
    return label

In [16]:
extractor = DatasetStepExtractorLoader.load('train_state2/dataset_extractor')

ValueError: too many values to unpack (expected 2)

In [ ]:
VOCAB_SIZE = extractor.dataset.phoneme.VOCAB_SIZE

In [ ]:
EMBEDDING_DIM = 256
UNITS = 1024
CODER_OUTPUT_DIM = 512
JOINT_NET_INNER_DIM = 1024

In [ ]:
transducer = Transducer(EMBEDDING_DIM, UNITS, CODER_OUTPUT_DIM, JOINT_NET_INNER_DIM, VOCAB_SIZE)
transducer.load_weights('./train_state2/checkpoints/ep0_no35168/ckpt')


In [ ]:
decoder = AudioDecoder(transducer)


In [ ]:
rst = decoder.decode(extractor.get_value()[0][0])
ret = []
for i in rst:
  ret.append(extractor.dataset.phoneme.phonemes[i])

print(ret)


['<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>', '<BLANK>']


In [ ]:
spectrogram, labels = extractor.get_value()[0]

logits = transducer((tf.convert_to_tensor([spectrogram], dtype=tf.float32), tf.convert_to_tensor([labels], dtype=tf.int32)))
print(tf.argmax(logits, axis=-1))
# print(logits[:, -30: , 0: ,])

tf.Tensor(
[[[0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 0 9 0 5 6 0 9 9 0]
  [0 6 0 

In [ ]:
def vector_to_sentence (x):
  return ''.join([extractor.dataset.phoneme.phonemes[idx] for idx in x][1: ])


In [ ]:
rst = []
x, y = 0, 0
while x < logits.shape[2] and y < logits.shape[1]:
  sel = tf.argmax(logits[0][y][x], axis=-1).numpy()
  rst.append(sel)
  if sel == 0:
    y += 1
  else:
    x += 1

print(rst)
print(vector_to_sentence([0] + list(filter(lambda x: x != 0, rst))))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



In [ ]:
print(moasseugi(vector_to_sentence(labels)))

멜로 멜로
